In [0]:
#!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#!apt-get update -qq 2>&1 > /dev/null
#!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


In [0]:
# Create a directory and mount Google Drive onto the colab workspace using that directory.

!mkdir -p drive

!google-drive-ocamlfuse drive
!pip install tables
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools


In [0]:
from keras.utils import np_utils 
from keras.datasets import mnist 
import seaborn as sns
from keras.initializers import RandomNormal

In [0]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import time
# https://gist.github.com/greydanus/f6eee59eaf1d90fcb3b534a25362cea4
# https://stackoverflow.com/a/14434334
# this function is used to update the plots for each epoch and error
def plt_dynamic(x, vy, ty, ax, colors=['b']):
    ax.plot(x, vy, 'b', label="Validation Loss")
    ax.plot(x, ty, 'r', label="Train Loss")
    plt.legend()
    plt.grid()
    fig.canvas.draw()

In [0]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
print("Number of training examples :", X_train.shape[0], "and each image is of shape (%d, %d)"%(X_train.shape[1], X_train.shape[2]))
print("Number of training examples :", X_test.shape[0], "and each image is of shape (%d, %d)"%(X_test.shape[1], X_test.shape[2]))

Number of training examples : 60000 and each image is of shape (28, 28)
Number of training examples : 10000 and each image is of shape (28, 28)


In [0]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1]*X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2]) 

In [0]:
print("Number of training examples :", X_train.shape[0], "and each image is of shape (%d)"%(X_train.shape[1]))
print("Number of training examples :", X_test.shape[0], "and each image is of shape (%d)"%(X_test.shape[1]))

Number of training examples : 60000 and each image is of shape (784)
Number of training examples : 10000 and each image is of shape (784)


In [0]:

print(X_train[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   3  18  18  18 126 136 175  26 166 255
 247 127   0   0   0   0   0   0   0   0   0   0   0   0  30  36  94 154
 170 253 253 253 253 253 225 172 253 242 195  64   0   0   0   0   0   0
   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251  93  82
  82  56  39   0   0   0   0   0   0   0   0   0   0   0   0  18 219 253
 253 253 253 253 198 182 247 241   0   0   0   0   

In [0]:
X_train1 = X_train/255
X_test1 = X_test/255

In [0]:
print(X_train1[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [0]:
# type(X_train)
# type(X_train1)
X_train1.shape

(60000, 784)

In [0]:
from sklearn.preprocessing import StandardScaler

sd = StandardScaler()
x_tr = sd.fit_transform(X_train)
# type(x_tr)
x_te = sd.fit_transform(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [0]:
# print(x_tr[0])
# type(x_tr)
x_tr.shape

(60000, 784)

In [0]:
from sklearn.preprocessing import normalize

x_tr1 = normalize(X_train)
x_te1 = normalize(X_test)

# type(x_tr1)
# x_tr1.shape
# print(x_tr1[0])

In [0]:
print("Class label of first image :", y_train[0])

# lets convert this into a 10 dimensional vector
# ex: consider an image is 5 convert it into 5 => [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
# this conversion needed for MLPs 

Y_train1 = np_utils.to_categorical(y_train, 10) 
Y_test1 = np_utils.to_categorical(y_test, 10)

print("After converting the output into a vector : ",Y_train1[0])

Class label of first image : 5
After converting the output into a vector :  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [0]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(n_values=10)

xr = y_train.reshape(-1,1)
xr1 = y_test.reshape(-1,1)

y_tr = ohe.fit_transform(xr)
y_te = ohe.fit_transform(xr1)
print(y_tr[0])

print(type(y_tr))

print(y_tr.shape)
# type(Y_train1)

  (0, 5)	1.0
<class 'scipy.sparse.csr.csr_matrix'>
(60000, 10)


In [0]:
from keras.models import Sequential 
from keras.layers import Dense, Activation 

output_dim = 10
input_dim = X_train1.shape[1]

batch_size = 128 
nb_epoch = 20

In [0]:
model = Sequential()
model.add(Dense(output_dim, input_dim=input_dim, activation='softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train1, Y_train1, batch_size = batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test1, Y_test1)) 



Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 4s 66us/step - loss: 1.2935 - acc: 0.6915 - val_loss: 0.8164 - val_acc: 0.8318
Epoch 2/20
60000/60000 [==============================] - 2s 38us/step - loss: 0.7168 - acc: 0.8410 - val_loss: 0.6086 - val_acc: 0.8592
Epoch 3/20
60000/60000 [==============================] - 2s 39us/step - loss: 0.5870 - acc: 0.8599 - val_loss: 0.5262 - val_acc: 0.8717
Epoch 4/20
60000/60000 [==============================] - 2s 40us/step - loss: 0.5251 - acc: 0.8691 - val_loss: 0.4802 - val_acc: 0.8785
Epoch 5/20
60000/60000 [==============================] - 3s 43us/step - loss: 0.4876 - acc: 0.8747 - val_loss: 0.4504 - val_acc: 0.8851
Epoch 6/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.4618 - acc: 0.8796 - val_loss: 0.4289 - val_acc: 0.8880
Epoch 7/20
60000/60000 [==============================] - 2s 40us/step - loss: 0.4426 - acc: 0.8827 - val_loss: 0.4130 - val_acc

In [0]:
model = Sequential()
model.add(Dense(output_dim, input_dim=input_dim, activation='softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_tr, Y_train1, batch_size = batch_size, epochs=nb_epoch, verbose=1, validation_data=(x_te, Y_test1)) 



Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 3s 46us/step - loss: 0.7212 - acc: 0.7956 - val_loss: 0.4372 - val_acc: 0.8797
Epoch 2/20
60000/60000 [==============================] - 3s 42us/step - loss: 0.4104 - acc: 0.8854 - val_loss: 0.3752 - val_acc: 0.8946
Epoch 3/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.3655 - acc: 0.8970 - val_loss: 0.3493 - val_acc: 0.9025
Epoch 4/20
60000/60000 [==============================] - 2s 40us/step - loss: 0.3433 - acc: 0.9035 - val_loss: 0.3356 - val_acc: 0.9062
Epoch 5/20
60000/60000 [==============================] - 2s 40us/step - loss: 0.3294 - acc: 0.9079 - val_loss: 0.3260 - val_acc: 0.9090
Epoch 6/20
60000/60000 [==============================] - 2s 39us/step - loss: 0.3196 - acc: 0.9110 - val_loss: 0.3187 - val_acc: 0.9114
Epoch 7/20
60000/60000 [==============================] - 2s 37us/step - loss: 0.3122 - acc: 0.9128 - val_loss: 0.3136 - val_acc

In [0]:
model = Sequential()
model.add(Dense(output_dim, input_dim=input_dim, activation='softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_tr1, Y_train1, batch_size = batch_size, epochs=nb_epoch, verbose=1, validation_data=(x_te1, Y_test1)) 



Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 3s 44us/step - loss: 2.2776 - acc: 0.2231 - val_loss: 2.2467 - val_acc: 0.3798
Epoch 2/20
60000/60000 [==============================] - 3s 45us/step - loss: 2.2194 - acc: 0.4597 - val_loss: 2.1890 - val_acc: 0.5318
Epoch 3/20
60000/60000 [==============================] - 3s 44us/step - loss: 2.1638 - acc: 0.5701 - val_loss: 2.1338 - val_acc: 0.6048
Epoch 4/20
60000/60000 [==============================] - 3s 44us/step - loss: 2.1104 - acc: 0.6228 - val_loss: 2.0808 - val_acc: 0.6492
Epoch 5/20
60000/60000 [==============================] - 3s 42us/step - loss: 2.0592 - acc: 0.6527 - val_loss: 2.0298 - val_acc: 0.6753
Epoch 6/20
60000/60000 [==============================] - 2s 38us/step - loss: 2.0100 - acc: 0.6739 - val_loss: 1.9810 - val_acc: 0.6977
Epoch 7/20
60000/60000 [==============================] - 2s 40us/step - loss: 1.9628 - acc: 0.6933 - val_loss: 1.9342 - val_acc

In [0]:
model = Sequential()
model.add(Dense(output_dim, input_dim=input_dim, activation='softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_tr, y_tr, batch_size = batch_size, epochs=nb_epoch, verbose=1, validation_data=(x_te, y_te)) 



Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 3s 46us/step - loss: 0.7281 - acc: 0.7914 - val_loss: 0.4370 - val_acc: 0.8794
Epoch 2/20
60000/60000 [==============================] - 3s 44us/step - loss: 0.4116 - acc: 0.8846 - val_loss: 0.3733 - val_acc: 0.8974
Epoch 3/20
60000/60000 [==============================] - 2s 42us/step - loss: 0.3647 - acc: 0.8973 - val_loss: 0.3478 - val_acc: 0.9033
Epoch 4/20
60000/60000 [==============================] - 2s 42us/step - loss: 0.3418 - acc: 0.9045 - val_loss: 0.3334 - val_acc: 0.9066
Epoch 5/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.3277 - acc: 0.9084 - val_loss: 0.3240 - val_acc: 0.9100
Epoch 6/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.3176 - acc: 0.9116 - val_loss: 0.3166 - val_acc: 0.9126
Epoch 7/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.3096 - acc: 0.9139 - val_loss: 0.3120 - val_acc

In [0]:
model = Sequential()
model.add(Dense(output_dim, input_dim=input_dim, activation='softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train1, y_tr, batch_size = batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test1, y_te)) 



Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 3s 45us/step - loss: 1.2704 - acc: 0.7103 - val_loss: 0.8037 - val_acc: 0.8348
Epoch 2/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.7102 - acc: 0.8430 - val_loss: 0.6031 - val_acc: 0.8634
Epoch 3/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.5834 - acc: 0.8606 - val_loss: 0.5227 - val_acc: 0.8767
Epoch 4/20
60000/60000 [==============================] - 2s 42us/step - loss: 0.5226 - acc: 0.8694 - val_loss: 0.4776 - val_acc: 0.8827
Epoch 5/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.4856 - acc: 0.8763 - val_loss: 0.4482 - val_acc: 0.8875
Epoch 6/20
60000/60000 [==============================] - 3s 42us/step - loss: 0.4601 - acc: 0.8811 - val_loss: 0.4273 - val_acc: 0.8904
Epoch 7/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.4413 - acc: 0.8845 - val_loss: 0.4112 - val_acc

In [0]:
score = model.evaluate(X_test1, Y_test1, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

fig,ax = plt.subplots(1,1)
ax.set_xlabel('epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')

# list of epoch numbers
x = list(range(1,nb_epoch+1))

# print(history.history.keys())
# dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])
# history = model_drop.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test, Y_test))

# we will get val_loss and val_acc only when you pass the paramter validation_data
# val_loss : validation loss
# val_acc : validation accuracy

# loss : training loss
# acc : train accuracy
# for each key in histrory.histrory we will have a list of length equal to number of epochs

vy = history.history['val_loss']
ty = history.history['loss']
plt_dynamic(x, vy, ty, ax)

Test score: 0.3353040454506874
Test accuracy: 0.908


<IPython.core.display.Javascript object>

In [0]:
score = model.evaluate(x_te, Y_test1, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

fig,ax = plt.subplots(1,1)
ax.set_xlabel('epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')

# list of epoch numbers
x = list(range(1,nb_epoch+1))

# print(history.history.keys())
# dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])
# history = model_drop.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test, Y_test))

# we will get val_loss and val_acc only when you pass the paramter validation_data
# val_loss : validation loss
# val_acc : validation accuracy

# loss : training loss
# acc : train accuracy
# for each key in histrory.histrory we will have a list of length equal to number of epochs

vy = history.history['val_loss']
ty = history.history['loss']
plt_dynamic(x, vy, ty, ax)

Test score: 0.2911778163164854
Test accuracy: 0.9222


<IPython.core.display.Javascript object>

In [0]:
score = model.evaluate(x_te1, Y_test1, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])


Test score: 1.4784557090759278
Test accuracy: 0.7915


In [0]:
# Multilayer perceptron

model_sigmoid = Sequential()
model_sigmoid.add(Dense(512, activation='sigmoid', input_shape=(input_dim,)))
model_sigmoid.add(Dense(128, activation='sigmoid'))
model_sigmoid.add(Dense(output_dim, activation='softmax'))

model_sigmoid.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_5 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1290      
Total params: 468,874
Trainable params: 468,874
Non-trainable params: 0
_________________________________________________________________


In [0]:
model_sigmoid.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_sigmoid.fit(X_train1, Y_train1, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test1, Y_test1))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 3s 48us/step - loss: 2.2729 - acc: 0.2132 - val_loss: 2.2238 - val_acc: 0.3349
Epoch 2/20
60000/60000 [==============================] - 3s 48us/step - loss: 2.1792 - acc: 0.4360 - val_loss: 2.1244 - val_acc: 0.5324
Epoch 3/20
60000/60000 [==============================] - 3s 51us/step - loss: 2.0617 - acc: 0.5580 - val_loss: 1.9800 - val_acc: 0.5790
Epoch 4/20
60000/60000 [==============================] - 3s 49us/step - loss: 1.8944 - acc: 0.6155 - val_loss: 1.7818 - val_acc: 0.6387
Epoch 5/20
60000/60000 [==============================] - 3s 47us/step - loss: 1.6796 - acc: 0.6579 - val_loss: 1.5493 - val_acc: 0.6988
Epoch 6/20
60000/60000 [==============================] - 3s 44us/step - loss: 1.4508 - acc: 0.6961 - val_loss: 1.3259 - val_acc: 0.7308
Epoch 7/20
60000/60000 [==============================] - 3s 45us/step - loss: 1.2470 - acc: 0.7348 - val_loss: 1.1408 - val_acc

In [0]:
model_sigmoid.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_sigmoid.fit(x_tr, Y_train1, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(x_te, Y_test1))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 3s 50us/step - loss: 0.4261 - acc: 0.8728 - val_loss: 0.3840 - val_acc: 0.8874
Epoch 2/20
60000/60000 [==============================] - 3s 47us/step - loss: 0.3844 - acc: 0.8860 - val_loss: 0.3668 - val_acc: 0.8927
Epoch 3/20
60000/60000 [==============================] - 3s 48us/step - loss: 0.3682 - acc: 0.8911 - val_loss: 0.3545 - val_acc: 0.8963
Epoch 4/20
60000/60000 [==============================] - 3s 49us/step - loss: 0.3556 - acc: 0.8958 - val_loss: 0.3441 - val_acc: 0.8992
Epoch 5/20
60000/60000 [==============================] - 3s 48us/step - loss: 0.3453 - acc: 0.8985 - val_loss: 0.3363 - val_acc: 0.9013
Epoch 6/20
60000/60000 [==============================] - 3s 48us/step - loss: 0.3366 - acc: 0.9014 - val_loss: 0.3286 - val_acc: 0.9036
Epoch 7/20
60000/60000 [==============================] - 3s 49us/step - loss: 0.3290 - acc: 0.9036 - val_loss: 0.3224 - val_acc

In [0]:
model_sigmoid.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_sigmoid.fit(x_tr1, Y_train1, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(x_te1, Y_test1))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 3s 53us/step - loss: 1.7655 - acc: 0.7810 - val_loss: 1.6896 - val_acc: 0.7618
Epoch 2/20
60000/60000 [==============================] - 3s 48us/step - loss: 1.6563 - acc: 0.8115 - val_loss: 1.6141 - val_acc: 0.7989
Epoch 3/20
60000/60000 [==============================] - 3s 48us/step - loss: 1.5851 - acc: 0.8123 - val_loss: 1.5438 - val_acc: 0.8594
Epoch 4/20
60000/60000 [==============================] - 3s 47us/step - loss: 1.5140 - acc: 0.8196 - val_loss: 1.4710 - val_acc: 0.8121
Epoch 5/20
60000/60000 [==============================] - 3s 47us/step - loss: 1.4420 - acc: 0.8259 - val_loss: 1.3975 - val_acc: 0.8336
Epoch 6/20
60000/60000 [==============================] - 3s 47us/step - loss: 1.3702 - acc: 0.8307 - val_loss: 1.3248 - val_acc: 0.8529
Epoch 7/20
60000/60000 [==============================] - 3s 47us/step - loss: 1.2996 - acc: 0.8363 - val_loss: 1.2573 - val_acc

In [0]:
model_sigmoid = Sequential()
model_sigmoid.add(Dense(512, activation='sigmoid', input_shape=(input_dim,)))
model_sigmoid.add(Dense(128, activation='sigmoid'))
model_sigmoid.add(Dense(output_dim, activation='softmax'))

model_sigmoid.summary()

model_sigmoid.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_sigmoid.fit(X_train1, Y_train1, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test1, Y_test1))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_11 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_12 (Dense)             (None, 10)                1290      
Total params: 468,874
Trainable params: 468,874
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 4s 64us/step - loss: 0.5487 - acc: 0.8544 - val_loss: 0.2524 - val_acc: 0.9259
Epoch 2/20
60000/60000 [==============================] - 3s 55us/step - loss: 0.2230 - acc: 0.9347 - val_loss: 0.1847 - val_acc: 0.9452
Epoch 3/20
60000/60000 [==============================] - 4s 58us/step - los

In [0]:
model_sigmoid = Sequential()
model_sigmoid.add(Dense(512, activation='sigmoid', input_shape=(input_dim,)))
model_sigmoid.add(Dense(128, activation='sigmoid'))
model_sigmoid.add(Dense(output_dim, activation='softmax'))

model_sigmoid.summary()

model_sigmoid.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_sigmoid.fit(x_tr, y_tr, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(x_te, y_te))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_14 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_15 (Dense)             (None, 10)                1290      
Total params: 468,874
Trainable params: 468,874
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 4s 68us/step - loss: 0.3995 - acc: 0.8946 - val_loss: 0.2138 - val_acc: 0.9378
Epoch 2/20
60000/60000 [==============================] - 4s 58us/step - loss: 0.1736 - acc: 0.9499 - val_loss: 0.1563 - val_acc: 0.9543
Epoch 3/20
60000/60000 [==============================] - 4s 59us/step - los

In [0]:
model_sigmoid = Sequential()
model_sigmoid.add(Dense(512, activation='sigmoid', input_shape=(input_dim,)))
model_sigmoid.add(Dense(128, activation='sigmoid'))
model_sigmoid.add(Dense(output_dim, activation='softmax'))

model_sigmoid.summary()

model_sigmoid.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_sigmoid.fit(X_train1, Y_train1, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test1, Y_test1))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_17 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_18 (Dense)             (None, 10)                1290      
Total params: 468,874
Trainable params: 468,874
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 4s 65us/step - loss: 0.5341 - acc: 0.8621 - val_loss: 0.2566 - val_acc: 0.9266
Epoch 2/20
60000/60000 [==============================] - 4s 58us/step - loss: 0.2221 - acc: 0.9341 - val_loss: 0.1866 - val_acc: 0.9445
Epoch 3/20
60000/60000 [==============================] - 4s 58us/step - los

In [0]:
model_sigmoid = Sequential()
model_sigmoid.add(Dense(512, activation='sigmoid', input_shape=(input_dim,)))
model_sigmoid.add(Dense(128, activation='sigmoid'))
model_sigmoid.add(Dense(output_dim, activation='softmax'))

model_sigmoid.summary()

model_sigmoid.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_sigmoid.fit(x_tr, Y_train1, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(x_te, Y_test1))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_20 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_21 (Dense)             (None, 10)                1290      
Total params: 468,874
Trainable params: 468,874
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 4s 68us/step - loss: 0.4034 - acc: 0.8929 - val_loss: 0.2125 - val_acc: 0.9385
Epoch 2/20
60000/60000 [==============================] - 4s 63us/step - loss: 0.1723 - acc: 0.9499 - val_loss: 0.1564 - val_acc: 0.9526
Epoch 3/20
60000/60000 [==============================] - 4s 63us/step - los

In [0]:
 from keras.optimizers import Adam,RMSprop,SGD
def best_hyperparameters(activ):

    model = Sequential()
    model.add(Dense(512, activation=activ, input_shape=(input_dim,), kernel_initializer=RandomNormal(mean=0.0, stddev=0.062, seed=None)))
    model.add(Dense(128, activation=activ, kernel_initializer=RandomNormal(mean=0.0, stddev=0.125, seed=None)) )
    model.add(Dense(output_dim, activation='softmax'))


    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    
    return model

In [0]:
# https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

activ = ['sigmoid','relu']

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=best_hyperparameters, epochs=nb_epoch, batch_size=batch_size, verbose=0)
param_grid = dict(activ=activ)

# if you are using CPU
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
# if you are using GPU dont use the n_jobs parameter

grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train1, Y_train1)

In [0]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.975967 using {'activ': 'relu'}
0.973917 (0.001068) with: {'activ': 'sigmoid'}
0.975967 (0.001915) with: {'activ': 'relu'}


In [0]:
# https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

activ = ['sigmoid','relu']

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=best_hyperparameters, epochs=nb_epoch, batch_size=batch_size, verbose=0)
param_grid = dict(activ=activ)

# if you are using CPU
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
# if you are using GPU dont use the n_jobs parameter

grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(x_tr, Y_train1)

In [0]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.966467 using {'activ': 'sigmoid'}
0.966467 (0.004873) with: {'activ': 'sigmoid'}
0.966417 (0.002044) with: {'activ': 'relu'}
